In [69]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [70]:
site = 'http://book.interpark.com'
url = f'{site}/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller'
req = requests.get(url)
html = req.text
#html

In [71]:
soup = BeautifulSoup(html, 'html.parser')
#soup

In [72]:
lis = soup.select('#content > div.rankBestWrapper > div.rankBestContainer > div.rankBestContents > div > div.rankBestContentList > ol > li')
len(lis)

15

In [73]:
li = lis[0]
href = li.find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [74]:
if len(li.select('.rankBtn_ctrl')) == 1:
    rank = str(li.select_one('.rankBtn_ctrl')['class'][1]).split('_B')[1]
else:
    rank = str(li.select('.rankBtn_ctrl')[0]['class'][1]).split('M0')[1]+str(li.select('.rankBtn_ctrl')[1]['class'][1]).split('M0')[1]


#li_10.select('.rankBtn_ctrl')[0]['class'][1]


In [75]:
name = li.select_one('div.itemName').get_text()
name

'트렌드 코리아 2022 '

In [76]:
author = li.select_one('span.author').get_text()
author

'김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영'

In [77]:
pub = li.select_one('span.company').get_text()
pub

'미래의창'

In [78]:
sub_url = site + href
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [79]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

In [91]:
fixed_price1 = sub_soup.select_one('p.fixedPrice').get_text()
fixed_price = int(fixed_price1.replace('원','').replace(',',''))
fixed_price

18000

In [92]:
price1 = sub_soup.select_one('span.price').get_text()
price = int(price1.replace(',',''))
price

16200

In [93]:
date1 = sub_soup.select_one('ul.bInfo_txt').get_text()
date = ' '.join(date1.strip().split()[-9:-6])
date

'2021년 09월 01일'

In [94]:
page1 = sub_soup.select_one('ul.bInfo_txt').get_text()
page = int(page1.strip().split()[-4])

In [95]:
lines = []

for li in lis:
    href = li.find('a')['href']
    
    if len(li.select('.rankBtn_ctrl')) == 1:
        rank = str(li.select_one('.rankBtn_ctrl')['class'][1]).split('_B')[1]
    else:
        rank = str(li.select('.rankBtn_ctrl')[0]['class'][1]).split('M0')[1] + str(li.select('.rankBtn_ctrl')[1]['class'][1]).split('M0')[1]
    

    name = li.select_one('div.itemName').get_text()
    author = li.select_one('span.author').get_text()
    pub = li.select_one('span.company').get_text()

    sub_url = site + href
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    fixed_price1 = sub_soup.select_one('p.fixedPrice').get_text()
    fixed_price = fixed_price1.replace('원','').replace(',','')
    price1 = sub_soup.select_one('span.price').get_text()
    price = int(price1.replace(',',''))
    date1 = sub_soup.select_one('ul.bInfo_txt').get_text()
    date = ' '.join(date1.strip().split()[-9:-6]) 
    page1 = sub_soup.select_one('ul.bInfo_txt').get_text()
    page = page1.strip().split()[-4] 

    lines.append([rank, name, author, pub, fixed_price, price, date, page])

In [96]:
df = pd.DataFrame(lines, columns=['순위', '책제목', '저자', '출판사', '정가', '할인가', '발행일', '쪽수'])
df

,순위,책제목,저자,출판사,정가,할인가,발행일,쪽수
0,01,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,18000,16200,2021년 10월 06일,452
1,02,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,47000,42300,2021년 11월 12일,1104
2,03,거인의 포트폴리오,강환국,페이지2북스,18000,16200,2021년 11월 08일,404
3,03,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,17500,15750,2021년 10월 29일,404
4,05,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,16000,14400,2021년 11월 12일,320
5,06,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,15000,13500,2021년 09월 29일,274
6,07,투자는 디테일에 있다,김정환,부케이,18000,16200,2021년 12월 10일,424
7,07,웰씽킹(Wealthinking),켈리 최,다산북스,16000,14400,2021년 11월 10일,316
8,07,오십에 읽는 논어,최종엽,유노북스,16000,14400,2021년 11월 03일,300
9,10,소크라테스 익스프레스(20만 부 기념 윈터 에디션),에릭 와이너/김하현 역,어크로스,18000,16200,2021년 11월 26일,524


In [97]:
df.to_csv('인터파크_베스트셀러_211206_16시.csv', index=False)